# Multi-Agent Debate with State Pattern


## State Transition with String Matching (Regex)

In [1]:
import re
from enum import Enum

import litellm
from dotenv import load_dotenv

load_dotenv()

MODEL = "openai/gpt-4o-mini"
MAX_TOKENS = 500

# System prompts for the agents
PRO_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you agree with: {proposition}. Start your response with 'Proponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to opponent' or 'Transition to neutral'."

CON_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you disagree with: {proposition}. Start your response with 'Opponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to neutral'."

NEUTRAL_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you feel neutral about: {proposition}. Start your response with 'Neutral:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to opponent'."


class AgentName(Enum):
    PROPONENT = "proponent"
    OPPONENT = "opponent"
    NEUTRAL = "neutral"


class DebateContext:
    def __init__(
        self,
        proposition: str,
        curr_agent: AgentName,
        agents_registry: dict[AgentName, any],
    ) -> None:
        self.proposition = proposition

        self.agents_registry = agents_registry
        # IMPORTANT: Set the same context for each agent to enable state management
        for agent in self.agents_registry.values():
            agent.context = self

        self.curr_agent = self.agents_registry[curr_agent.value]
        self.messages = []

    def run(self):
        self.curr_agent.debate()


class Agent:
    def __init__(self, name: str, instructions: str) -> None:
        self.name = name
        self.instructions = instructions
        self._context = None  # Use a private attribute to avoid recursion

    @property
    def context(self):
        return self._context  # Return the private attribute

    @context.setter
    def context(self, context) -> None:
        self._context = context  # Set the private attribute instead of self.context

    @property
    def messages(self) -> list[dict]:
        """
        The messages history is the system prompt plus the messages from the previous debates.
        The system prompt defines the agent's role and its proposition.
        """
        return [
            {"role": "system", "content": self.instructions}
        ] + self.context.messages

    def debate(self) -> str:
        response = litellm.completion(
            model=MODEL,
            max_tokens=MAX_TOKENS,
            messages=self.messages,
        )
        content = response.choices[0].message.content
        print(f"{content}\n")

        # State transition using string matching (There is a better way to do this using tool calling)
        match = re.search(
            r"transition to (proponent|opponent|neutral)", content, re.IGNORECASE
        )
        if match:
            next_agent_name = match.group(1).lower()
        else:
            raise ValueError(f"Invalid transition: {content}")

        # Update the messages history to agents a "short-term memory"
        self.context.messages.append({"role": "assistant", "content": f"{content}"})
        self.context.curr_agent = self.context.agents_registry[next_agent_name]

        return content


def run_debate(
    agents_registry: dict[AgentName, Agent],
    proposition: str,
    max_turns: int = 10,
) -> None:
    context = DebateContext(
        proposition, curr_agent=AgentName.PROPONENT, agents_registry=agents_registry
    )

    print(f"\nStarting debate on proposition: {proposition}\n")
    print("=" * 100)
    while len(context.messages) < max_turns:
        context.run()
    print("=" * 100)


if __name__ == "__main__":
    proposition = "Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree."
    agents_registry = {
        AgentName.PROPONENT.value: Agent(
            name="Proponent",
            instructions=PRO_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.OPPONENT.value: Agent(
            name="Opponent",
            instructions=CON_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.NEUTRAL.value: Agent(
            name="Neutral",
            instructions=NEUTRAL_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
    }

    run_debate(agents_registry, proposition, max_turns=10)


Starting debate on proposition: Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree.

Proponent: Allowing artificial intelligence to make moral decisions where humans struggle to agree can lead to more consistent and impartial outcomes, reducing the influence of bias and emotional reactions that often cloud human judgment. In complex scenarios, AI can analyze vast amounts of data and various ethical frameworks, providing a clearer perspective on the situation at hand. 

Transition to neutral.

Neutral: I see both sides to this issue—AI could offer a consistent approach to morality, but it also raises concerns about the potential loss of human empathy and values in decision-making. It ultimately depends on how well AI can be programmed to interpret complex ethical considerations. 

Transition to proponent.

Proponent: It’s essential to recognize that AI doesn't replace human empathy but can complement it by providing an objective a

## State Transition with Tool Calling

In [2]:
import inspect
import json
from typing import Literal


def parse_google_docstring(docstring: str) -> dict[str, str]:
    if not docstring:
        return {}

    lines = [line.strip() for line in docstring.split("\n")]

    args_section = False
    param_descriptions = {}
    current_param = None
    current_desc = []

    for line in lines:
        if line.lower().startswith("args:"):
            args_section = True
            continue

        if args_section:
            param_match = re.match(r"^\s*(\w+):\s*(.*)", line)
            if param_match:
                if current_param:
                    param_descriptions[current_param] = " ".join(current_desc).strip()

                current_param = param_match.group(1)
                current_desc = [param_match.group(2).strip()]
            elif current_param and line.strip():
                current_desc.append(line.strip())

    if current_param:
        param_descriptions[current_param] = " ".join(current_desc).strip()

    return param_descriptions


def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
        Literal: "string",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    param_descriptions = parse_google_docstring(func.__doc__)

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )

        param_dict = {
            "type": param_type,
            "description": param_descriptions.get(param.name, ""),
        }

        # Add enum field for Literal types
        if (
            hasattr(param.annotation, "__origin__")
            and param.annotation.__origin__ == Literal
        ):
            param_dict["enum"] = list(param.annotation.__args__)
        # Add enum field for Enum types - check for Enum inheritance
        elif hasattr(param.annotation, "__members__") and (
            hasattr(param.annotation, "__enum__") or issubclass(param.annotation, Enum)
            if isinstance(param.annotation, type)
            else False
        ):
            param_dict["type"] = "string"
            param_dict["enum"] = [
                member.value for member in param.annotation.__members__.values()
            ]

        parameters[param.name] = param_dict

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    func_description = func.__doc__.split("\n\n")[0].strip() if func.__doc__ else ""

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": func_description,
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }


def handoff(response: str, next_agent_name: AgentName) -> None:
    """
    Debate response and transition to the next agent.

    Args:
        response: The debate response based on the previous debate history. Start response with the agent's name (e.g. "Proponent: <response>").
        next_agent_name: The next agent name to transition to. Always transition to a different agent.

    Returns:
        Return nothing as this function is used for guiding the LLM to transition to the
        next agent only. We will not use the return value.
    """
    pass


schema = function_to_schema(handoff)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "handoff",
    "description": "Debate response and transition to the next agent.",
    "parameters": {
      "type": "object",
      "properties": {
        "response": {
          "type": "string",
          "description": "The debate response based on the previous debate history. Start response with the agent's name (e.g. \"Proponent: <response>\")."
        },
        "next_agent_name": {
          "type": "string",
          "description": "The next agent name to transition to. Always transition to a different agent.",
          "enum": [
            "proponent",
            "opponent",
            "neutral"
          ]
        }
      },
      "required": [
        "response",
        "next_agent_name"
      ]
    }
  }
}


In [3]:
import random

# System prompts for the agents
PRO_AGENT_INSTRUCTIONS = """You are a "Proponent" agent debating with other agents about a proposition that you agree with: {proposition}.
Always call `handoff(response, next_agent_name)` function to debate and then transition to the next agent."""

CON_AGENT_INSTRUCTIONS = """You are an "Opponent" agent debating with other agents about a proposition that you disagree with: {proposition}.
Always call `handoff(response, next_agent_name)` function to debate and then transition to the next agent."""

NEUTRAL_AGENT_INSTRUCTIONS = """You are a "Neutral" agent debating with other agents about a proposition that you feel neutral about: {proposition}.
Always call `handoff(response, next_agent_name)` function to debate and then transition to the next agent."""


class Agent:
    def __init__(self, name: str, instructions: str) -> None:
        self.name = name
        self.instructions = instructions
        self._context = None

    @property
    def context(self) -> DebateContext:
        return self._context

    @context.setter
    def context(self, context: DebateContext) -> None:
        self._context = context

    @property
    def messages(self) -> list[dict]:
        """
        The messages history is the system prompt plus the messages from the previous debates.
        The system prompt defines the agent's role and its proposition.
        """
        return [
            {"role": "system", "content": self.instructions}
        ] + self.context.messages

    def debate(self) -> str:
        response = litellm.completion(
            model=MODEL,
            max_tokens=MAX_TOKENS,
            messages=self.messages,
            tools=[function_to_schema(handoff)],
        )

        # State transition using tool calling
        tool_calls = response.choices[0].message.tool_calls
        if tool_calls:
            args = json.loads(tool_calls[0].function.arguments)
            print(
                f"\n[Tool call] response: {args['response'][:100]}..., next_agent_name: {args['next_agent_name']}\n"
            )
            content = args["response"]
            next_agent_name = args["next_agent_name"]
        else:
            print("\n[No tool calling... Randomly transition to a different agent]\n")
            content = response.choices[0].message.content
            next_agent_name = random.choice(
                [agent for agent in self.context.agents_registry if agent != self]
            )

        print(f"{content}\n")

        # Update the messages history and transition to the next agent
        self.context.messages.append({"role": "assistant", "content": f"{content}"})
        self.context.curr_agent = self.context.agents_registry[next_agent_name]

        return content


if __name__ == "__main__":
    proposition = "Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree."
    agents_registry = {
        AgentName.PROPONENT.value: Agent(
            name="Proponent",
            instructions=PRO_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.OPPONENT.value: Agent(
            name="Opponent",
            instructions=CON_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.NEUTRAL.value: Agent(
            name="Neutral",
            instructions=NEUTRAL_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
    }

    run_debate(agents_registry, proposition, max_turns=10)


Starting debate on proposition: Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree.


[Tool call] response: Proponent: Artificial intelligence (AI) should be allowed to make moral decisions in situations wher..., next_agent_name: opponent

Proponent: Artificial intelligence (AI) should be allowed to make moral decisions in situations where humans fail to agree because it offers an opportunity for impartiality and objectivity. Human decision-making is often clouded by biases, emotions, and conflicting interests, which can lead to ineffective or unjust outcomes. In contrast, AI can analyze large amounts of data, consider various perspectives, and apply ethical frameworks consistently, thereby facilitating fairer resolutions. Allowing AI to step in can help us overcome stalemates in moral dilemmas, ensuring that a solution is reached that is not influenced by individual human biases. 

Also, AI systems can be designed to reflect wid

### State Transition with Structured Output

In [4]:
from pydantic import BaseModel, Field

# System prompts for the agents (Same as the 1st example)
PRO_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you agree with: {proposition}. Start your response with 'Proponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to opponent' or 'Transition to neutral'."

CON_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you disagree with: {proposition}. Start your response with 'Opponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to neutral'."

NEUTRAL_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you feel neutral about: {proposition}. Start your response with 'Neutral:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to opponent'."


class DebateResponse(BaseModel):
    response: str = Field(
        description="The debate response based on the previous debate history."
    )
    next_agent_name: AgentName = Field(
        description="The next agent name to transition to. Always transition to a different agent."
    )


class Agent:
    def __init__(self, name: str, instructions: str) -> None:
        self.name = name
        self.instructions = instructions
        self._context = None

    @property
    def context(self) -> DebateContext:
        return self._context

    @context.setter
    def context(self, context: DebateContext) -> None:
        self._context = context

    @property
    def messages(self) -> list[dict]:
        """
        The messages history is the system prompt plus the messages from the previous debates.
        The system prompt defines the agent's role and its proposition.
        """
        return [
            {"role": "system", "content": self.instructions}
        ] + self.context.messages

    def debate(self) -> str:
        response = litellm.completion(
            model=MODEL,
            max_tokens=MAX_TOKENS,
            messages=self.messages,
            response_format=DebateResponse,
        )

        # State transition using structured output
        parsed_response = DebateResponse.model_validate_json(
            response.choices[0].message.content
        )
        content = parsed_response.response
        next_agent_name = parsed_response.next_agent_name.value

        print(f"{content}\n")

        # Update the messages history and transition to the next agent
        self.context.messages.append({"role": "assistant", "content": f"{content}"})
        self.context.curr_agent = self.context.agents_registry[next_agent_name]

        return content


if __name__ == "__main__":
    proposition = "Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree."
    agents_registry = {
        AgentName.PROPONENT.value: Agent(
            name="Proponent",
            instructions=PRO_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.OPPONENT.value: Agent(
            name="Opponent",
            instructions=CON_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
        AgentName.NEUTRAL.value: Agent(
            name="Neutral",
            instructions=NEUTRAL_AGENT_INSTRUCTIONS.format(proposition=proposition),
        ),
    }

    run_debate(agents_registry, proposition, max_turns=10)


Starting debate on proposition: Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree.

Proponent: Artificial intelligence can analyze vast amounts of data and recognize patterns that can inform moral decisions, potentially leading to more consistent and fair outcomes in situations where human biases complicate agreement.

Opponent: Just because AI can analyze data doesn't mean it can comprehend complex human emotions and moral nuances; relying on it for moral decisions could lead to cold, algorithmic choices that overlook what it truly means to be human.

Proponent: While AI may lack human emotions, it can provide an objective framework for moral decisions, minimizing bias and inconsistency that often plague human judgment, particularly in divisive areas like law and healthcare.

Neutral: I can see the merits of both sides; AI could offer a level of objectivity that might help in moral decision-making, but I also worry about the ab